In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load CSV file
df = pd.read_csv('english_to_spanish.csv')
english_sentences = df['english'].tolist()
spanish_sentences = df['spanish'].tolist()

# Tokenize sentences using TensorFlow
tokenizer_en = tf.keras.preprocessing.text.Tokenizer(filters='', lower=True)
tokenizer_es = tf.keras.preprocessing.text.Tokenizer(filters='', lower=True)

tokenizer_en.fit_on_texts(english_sentences)
tokenizer_es.fit_on_texts(spanish_sentences)

# Add special tokens to the tokenizer's vocabulary
special_tokens = ['<pad>', '<sos>', '<eos>', '<unk>']
for token in special_tokens:
    tokenizer_en.word_index[token] = len(tokenizer_en.word_index) + 1
    tokenizer_es.word_index[token] = len(tokenizer_es.word_index) + 1

vocab_size_en = len(tokenizer_en.word_index) + 1
vocab_size_es = len(tokenizer_es.word_index) + 1

# Convert sentences to sequences of integers
english_sequences = tokenizer_en.texts_to_sequences(english_sentences)
spanish_sequences = tokenizer_es.texts_to_sequences(spanish_sentences)

# Add <sos> and <eos> tokens
english_sequences = [[tokenizer_en.word_index['<sos>']] + seq + [tokenizer_en.word_index['<eos>']] for seq in english_sequences]
spanish_sequences = [[tokenizer_es.word_index['<sos>']] + seq + [tokenizer_es.word_index['<eos>']] for seq in spanish_sequences]

# Pad sequences
max_length_en = max(len(seq) for seq in english_sequences)
max_length_es = max(len(seq) for seq in spanish_sequences)

english_padded = tf.keras.preprocessing.sequence.pad_sequences(english_sequences, maxlen=max_length_en + 2, padding='post', value=tokenizer_en.word_index['<pad>'])
spanish_padded = tf.keras.preprocessing.sequence.pad_sequences(spanish_sequences, maxlen=max_length_es + 2, padding='post', value=tokenizer_es.word_index['<pad>'])

# Split the dataset into training and validation sets
train_data_es, val_data_es, train_data_en, val_data_en = train_test_split(spanish_padded, english_padded, test_size=0.1, random_state=42)

# Convert to PyTorch tensors
train_data_es = torch.tensor(train_data_es, dtype=torch.long)
val_data_es = torch.tensor(val_data_es, dtype=torch.long)
train_data_en = torch.tensor(train_data_en, dtype=torch.long)
val_data_en = torch.tensor(val_data_en, dtype=torch.long)

# Encoding and decoding functions for integer sequences to sentences and vice versa
def encode_sentence(sentence, tokenizer, add_special_tokens=True):
    tokens = tokenizer.texts_to_sequences([sentence.lower()])[0]
    if add_special_tokens:
        tokens = [tokenizer.word_index['<sos>']] + tokens + [tokenizer.word_index['<eos>']]
    return tokens

def decode_sequence(sequence, tokenizer):
    reverse_word_index = {index: word for word, index in tokenizer.word_index.items()}
    decoded_sentence = ' '.join([reverse_word_index.get(i, '<unk>') for i in sequence if i not in [tokenizer.word_index['<pad>'], tokenizer.word_index['<sos>'], tokenizer.word_index['<eos>']]])
    return decoded_sentence

# Hyperparameters
batch_size = 2
block_size = max(max_length_en, max_length_es) + 2  # account for <sos> and <eos>
max_iters = 100
eval_interval = 10
learning_rate = 1e-3
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

torch.manual_seed(1337)

# Data loading
def get_batch(split):
    if split == 'train':
        data_en, data_es = train_data_en, train_data_es
    else:
        data_en, data_es = val_data_en, val_data_es

    ix = torch.randint(len(data_en), (batch_size,))
    x = torch.stack([data_en[i] for i in ix])
    y = torch.stack([data_es[i] for i in ix])
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """One head of self-attention"""

    def __init__(self, head_size, mask_future=True):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        self.mask_future = mask_future

    def forward(self, x, context=None):
        B, T, C = x.shape
        k = self.key(x if context is None else context)
        q = self.query(x)
        v = self.value(x if context is None else context)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5
        if self.mask_future:
            wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        out = wei @ v
        return out

class SelfAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size, mask_future=True):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, mask_future) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class CrossAttention(nn.Module):
    """Multiple heads of cross-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, mask_future=False) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, context):
        out = torch.cat([h(x, context) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class EncoderBlock(nn.Module):
    """Transformer encoder block"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.self_attn = SelfAttention(n_head, head_size, mask_future=False)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.self_attn(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class DecoderBlock(nn.Module):
    """Transformer decoder block"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.self_attn = SelfAttention(n_head, head_size, mask_future=True)
        self.cross_attn = CrossAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ln3 = nn.LayerNorm(n_embd)

    def forward(self, x, encoder_output):
        x = x + self.self_attn(self.ln1(x))
        x = x + self.cross_attn(self.ln2(x), encoder_output)
        x = x + self.ffwd(self.ln3(x))
        return x

class TransformerLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table_en = nn.Embedding(vocab_size_en, n_embd)
        self.token_embedding_table_es = nn.Embedding(vocab_size_es, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.encoder_blocks = nn.Sequential(*[EncoderBlock(n_embd, n_head) for _ in range(n_layer)])
        self.decoder_blocks = nn.Sequential(*[DecoderBlock(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size_es)

    def forward(self, src, tgt=None):
        B, T_src = src.shape
        tok_emb_src = self.token_embedding_table_en(src)
        pos_emb_src = self.position_embedding_table(torch.arange(T_src, device=src.device))
        x = tok_emb_src + pos_emb_src
        encoder_output = self.encoder_blocks(x)

        if tgt is not None:
            T_tgt = tgt.shape[1]
            tok_emb_tgt = self.token_embedding_table_es(tgt)
            pos_emb_tgt = self.position_embedding_table(torch.arange(T_tgt, device=tgt.device))
            decoder_input = tok_emb_tgt + pos_emb_tgt
            decoder_output = decoder_input
            for decoder_block in self.decoder_blocks:
                decoder_output = decoder_block(decoder_output, encoder_output)
            x = decoder_output

        logits = self.lm_head(x)
        loss = None
        if tgt is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            tgt = tgt.view(B * T)
            loss = F.cross_entropy(logits, tgt, ignore_index=0)  # ignore padding index
        return logits, loss

    def generate(self, src, max_new_tokens):
        for _ in range(max_new_tokens):
            src_cond = src[:, -block_size:]
            logits, _ = self(src_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            src_next = torch.multinomial(probs, num_samples=1)
            src = torch.cat((src, src_next), dim=1)
        return src

model = TransformerLanguageModel()

# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# context = torch.zeros((1, 1), dtype=torch.long)
# print(decode_sequence(model.generate(context, max_new_tokens=500)[0].tolist(), tokenizer_es))




0
step 0: train loss 11.1831, val loss 11.1806
1
2
3
4
5
6
7
8
9
10
step 10: train loss 2.0920, val loss 2.0215
11
12
13
14
15
16
17
18
19
20
step 20: train loss 1.0396, val loss 1.0560
21
22
23
24
25
26
27
28
29
30
step 30: train loss 0.9688, val loss 1.0079
31
32
33
34
35
36
37
38
39
40
step 40: train loss 0.9244, val loss 0.9283
41
42
43
44
45
46
47
48
49
50
step 50: train loss 0.8583, val loss 0.8307
51
52
53
54
55
56
57
58
59
60
step 60: train loss 0.8244, val loss 0.8127
61
62
63
64
65
66
67
68
69
70
step 70: train loss 0.7993, val loss 0.8009
71
72
73
74
75
76
77
78
79
80
step 80: train loss 0.7760, val loss 0.7638
81
82
83
84
85
86
87
88
89
90
step 90: train loss 0.7183, val loss 0.7407
91
92
93
94
95
96
97
98
99


In [3]:
def test_model(input_sentence, model, tokenizer_en, tokenizer_es):
   
    input_sequence = encode_sentence(input_sentence, tokenizer_en)
    input_tensor = torch.tensor([input_sequence], dtype=torch.long)
    
    
    with torch.no_grad():
        output_logits, _ = model(input_tensor)
    
    predicted_sequence = torch.argmax(output_logits, dim=-1).squeeze().tolist()
    predicted_sentence = decode_sequence(predicted_sequence, tokenizer_es)
    
    return predicted_sentence


input_sentence = "Hello"


predicted_spanish_sentence = test_model(input_sentence, model, tokenizer_en, tokenizer_es)
print("Predicted Spanish sentence:", predicted_spanish_sentence)




Predicted Spanish sentence: parece silbes podemos
